<a href="https://colab.research.google.com/github/seojeongyun/Word2Vec/blob/main/word2vec_for_korean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/My Drive/ratings_train.txt"

[Errno 20] Not a directory: '/content/drive/My Drive/ratings_train.txt'
/content


In [ ]:
!git clone https://github.com/seojeongyun/naver_movie_review_sentiment_analysis

Cloning into 'naver_movie_review_sentiment_analysis'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 3), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 8.09 MiB | 1.52 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
import urllib

if __name__ == '__main__':
    # Data download from url
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                               filename="/content/drive/My Drive/ratings_train.txt")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                               filename="/content/drive/My Drive/ratings_test.txt")

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 50.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import urllib
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt

class preprocess:
    def __init__(self):
        # By using pandas, data save
        self.train_data = pd.read_table('/content/drive/My Drive/ratings_train.txt')
        self.test_data = pd.read_table('/content/drive/My Drive/ratings_test.txt')
        self.stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
        self.Okt = Okt()

    def dataset_len(self, type: str):  # Check the number of datasets
        if type == 'train':
            print('The number of train reviews : ', len(self.train_data))
        else:
            print('The number of test reviews : ', len(self.test_data))

    def process(self, type: str):
        if type == 'train':
            # Remove duplications column of document
            self.train_data.drop_duplicates(subset=['document'], inplace=True)

            # Remove the samples with null value
            train_data = self.train_data.dropna(how='any')

            # Remove special characters with regular expression
            train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
            train_data[:5]

            # Change the white space value to Null value and then remove
            train_data['document'] = train_data['document'].str.replace('^ +', "")  # change the white space to empty value
            train_data['document'].replace('', np.nan, inplace=True)
            print(train_data.isnull().sum())
            train_data = train_data.dropna(how='any')

            y_train = np.array(train_data['label']) # get labels
            return train_data, y_train

        else:
            # Apply same preprocess to test dataset
            self.test_data.drop_duplicates(subset=['document'], inplace=True)

            # Remove the samples with null value
            test_data = self.train_data.dropna(how='any')

            # remove duplicate value for column of ducument
            test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
            test_data[:5]

            test_data['document'] = self.test_data['document'].str.replace('^ +', "")  # change the white space to empty value
            test_data['document'].replace('', np.nan, inplace=True)  # change epmty space to null value
            test_data = test_data.dropna(how='any')  # remove the null value
            y_test = np.array(test_data['label'])
            return test_data, y_test

    def check_duplication(self):  # Check duplications column of document and label
        self.train_data['document'].nunique(), self.train_data['label'].nunique()
        self.test_data['document'].nunique(), self.test_data['label'].nunique()

    def remove_stopword(self, type: str, data):
        if type == 'train':
            X_train = []
            for sentence in tqdm(data['document']):
                tokenized_sentence = (self.Okt.morphs(sentence, stem=True))  # 토큰화
                stopwords_removed_sentence = [word for word in tokenized_sentence if not word in self.stopwords]  # 불용어 제거
                X_train.append(stopwords_removed_sentence)
            return X_train

        else:
            X_test = []
            for sentence in tqdm(data['document']):
                tokenized_sentence = self.Okt.morphs(sentence, stem=True)  # 토큰화
                stopwords_removed_sentence = [word for word in tokenized_sentence if not word in self.stopwords]  # 불용어 제거
                X_test.append(stopwords_removed_sentence)
            return X_test

# if __name__ == '__main__':
#     data = preprocess()

#     data.dataset_len('train')
#     data.dataset_len('test')

#     data.check_duplication()

#     data.process('train')
#     data.process('test')

#     # Check the ratio of train and labels
#     data.train_data['label'].value_counts()

#     # Check the null value from train set
#     print(data.train_data.isnull().values.any())
#     print(data.train_data.isnull().sum())
#     data.train_data.loc[data.train_data.document.isnull()]

#     print('The number of test dataset after preprocess :',len(data.test_data))

In [ ]:
if __name__ == '__main__':
    ### 클래스 선언
    token = Tokenize()
    data = preprocess()

In [ ]:
if __name__ == '__main__':
    ### 데이터 전처리
    X_train, Y_train = data.process('train')
    X_test, Y_test = data.process('test')

    ## 불용어 제거
    X_train_removed = data.remove_stopword('train', X_train)
    X_test_removed = data.remove_stopword('test', X_test)

<ipython-input-7-4fa79e7dfdbd>:35: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-7-4fa79e7dfdbd>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-7-4fa79e7dfdbd>:39: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "")  # change the white space to empty value
<ipython-input-7-4fa79e7dfdbd>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

id            0
document    789
label         0
dtype: int64


<ipython-input-7-4fa79e7dfdbd>:55: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
<ipython-input-7-4fa79e7dfdbd>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
<ipython-input-7-4fa79e7dfdbd>:58: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = self.test_data['document'].str.replace('^ +', "")  # change the white space to empty value
<ipython-input-7-4fa79e7dfdbd>:58: SettingWithCopyWarning: 
A value is try

단어 집합(vocabulary)의 크기 : 43752
등장 빈도가 2번 이하인 희귀 단어의 수: 24337
단어 집합에서 희귀 단어의 비율: 55.62488571950996
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.0
단어 집합의 크기 : 19416
145393
###############################
vocabsize로 토크나이저설정
핏온텍스트


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



핏온텍스트


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = X_train_removed, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
print(model.wv.most_similar("최민식"))